In [1]:
using General.Aux

**[@. macro, but partly escape](https://discourse.julialang.org/t/broadcast-macro-escape-from-it/94396)** <br>


@. sqrt(abs($sort(x))) is equivalent to <br>
sqrt.(abs.(sort(x))) , no  dot for sort.<br><br>
  @. is equivalent to a call to @__dot__. <br><br>

help?> @.<br>
should have read the help indeed

In [1]:
x = [1, 2, 3]; y = [10, 20, 30]
@. x + 3 * y; b = 2x + y

@logt x y b

@. a = x + 3 * y; b = 2x + y

LoadError: LoadError: UndefVarError: `@logt` not defined
in expression starting at e:\JuliaProjects\Training.jl\chp02\broadcast3_@..ipynb:4

In [ ]:
a = [1, 2, 3]; b = [10, 20, 30]; c= [0.1, 0.2, 0.3]

s1 = sum([1, 2, 3])
s2 = sum.([1, 2, 3])

result1 = @. sum(a + b + c)

result2 = @. $sum(a + b + c)

@logt s1 s2 result1 result2

[Broadcasting in Julia: the Good, the Bad, and the Ugly](https://bkamins.github.io/julialang/2022/06/24/broadcasting.html)

In [ ]:
x = [1, 2, 3]

r3 = sin.(x).^2 .+ cos.(x).^2

r4 = @. (sin(x)^2 + cos(x)^2)

r5 = @. sin(x)^2 + cos(x)^2

@logt r3 r4 r5

In [ ]:
using Statistics

In [ ]:
# compute variance of x
v1 = sum((x .- mean(x)) .^ 2) / (length(x) - 1)

v2 = @. $/($sum((x - $mean(x)) ^ 2), $-($length(x), 1))

v3 = @. /($sum((x - $mean(x)) ^ 2), $-($length(x), 1)) # 去掉第1个$不如上行清楚

@logt v1 v2 v3

As you can see I use broadcasting in only two places, while most of the operations are not broadcasted. If we wanted to use @. macro we would need to use $ escaping, which is equivalent and ugly. You can check it by using @macroexpand

In [ ]:
@macroexpand @. $/($sum((x - $mean(x))^2), $-($length(x), 1))

In [ ]:
@. $sum((x - $mean(x))^2) / ($length(x) - 1)

However, in this case you need to know and be sure that by not escaping-out the / and - function calls in the second part of the expression you will not affect the correctness of your calculation.<br><br>
In summary, I do not use @. in complex expressions as it is usually hard to reason about it.<br><br>
Let us now switch to some special cases of using @..

**Be careful with broadcasted assignment**

In [ ]:
x = ["a", "b"]
x = @. length(x)

In [ ]:
y = ["a", "b"]
@. y = length(y)

But, Vector{Char}, as Char supports conversion from integer:

In [ ]:
z = ['a', 'b']

In [ ]:
@. z = length(z)

In [ ]:
z

**Incorrect handling of named tuples**

In [ ]:
v = 1:3
[x in (a=1, c=3) for x in v]

In [ ]:
# We can rewrite it using broadcasting as:
in.(v, Ref((a=1, c=3)))

In [ ]:
# Now we think we could use the @. here as follows:
@. in(v, $Ref((a=1, c=3)))

This fails as @. macro incorrectly handles = inside NamedTuple definition. We have to write:

In [ ]:
@. in(v, $Ref((; a=1, c=3)))

The ; at the beginning of NamedTuple definition gives an equivalent object but changes how the code expression is transformed by the Julia compiler and it works. Here is how we can check the difference in the representation of **(a=1, c=3) and (; a=1, c=3):**

In [ ]:
dump(:(a=1, c=3))

In [ ]:
dump(:(; a=1, c=3))

Fortunately the case of NamedTuple is not likely to be problematic in practice as it is extremely rare.

**Conclusions**<br><br>
The @. macro can be very convenient. However, in my experience, you need to be careful when you use it as it is easy to get surprising results if you work with complex expressions. Out of the possible problematic situations I have covered in my post a most common one is forgetting to add $ to avoid broadcasting of some function calls in a complex expression.

In [2]:
x = [1, 2, 3]; y = [10, 20, 30]
@. x + 3 * y; b = 2x + y

@logt x y b

@. a = x + 3 * y; b = 2x + y


x, Vector{Int64}
  = [1, 2, 3]

y, Vector{Int64}
  = [10, 20, 30]

b, Vector{Int64}
  = [12, 24, 36]


UndefVarError: UndefVarError: `a` not defined

In [4]:
x = [1, 2, 3]; y = [10, 20, 30]

@.; a = x + 3 * y; b = 2x + y

@logt a b

LoadError: LoadError: MethodError: no method matching var"@__dot__"(::LineNumberNode, ::Module)

Closest candidates are:
  var"@__dot__"(::LineNumberNode, ::Module, !Matched::Any)
   @ Base broadcast.jl:1319

in expression starting at e:\JuliaProjects\Training3.jl\macro3\macro3_1.ipynb:3

In [22]:
x = [1, 2, 3]; y = [10, 20, 30]

@. :a = x + 3 * y; b = 2x + y  # $a, esc(a), :a都不行

MethodError: MethodError: no method matching ndims(::Type{Symbol})

Closest candidates are:
  ndims(!Matched::Type{Union{}}, Any...)
   @ Base abstractarray.jl:276
  ndims(!Matched::Type{<:Ref})
   @ Base refpointer.jl:96
  ndims(!Matched::Type{<:Number})
   @ Base number.jl:86
  ...


In [23]:
x = [1, 2, 3]; y = [10, 20, 30]

a = @.(x + 3 * y); b = @.(2x + y)

@logt a b


a, Vector{Int64}
  = [31, 62, 93]

b, Vector{Int64}
  = [12, 24, 36]
